# Daily AmMoneyline Monitoring

In [8]:
import io
import os
import openpyxl
import glob as glob
import pandas as pd
import zipfile as zf
import ipywidgets as widgets

from ipywidgets import interactive

SRC=r'\\ambankgroup.ahb.com\GFS\DPM-Performance Management\Historical Data\Raw Data\Retail Analytic\AML'
INFILE = r'\INFILE'
OUTFILE = r'\OUTFILE'

fy = "FY 23/24"


In [9]:
file_pattern = "MIS23110545_AML_Monitoring_Report_"

all_zips = glob.glob(os.path.join(SRC, f'{file_pattern}*.zip'))
all_zips.sort()

## OPTION 1: Select file from list

In [10]:
#list all available target file, select from dropdown
list_files=[]
def update_dropdown(list_files):
    display(f"{list_files}")

for f in all_zips:
    list_files.append(os.path.basename(f))

file_select=widgets.Dropdown(options=list_files, description=os.path.basename(SRC))
display(interactive(update_dropdown, list_files=file_select))
#file_select.value

interactive(children=(Dropdown(description='AML', options=('MIS23110545_AML_Monitoring_Report_20231115.zip', '…

In [12]:
zip_file=os.path.join(SRC, file_select.value)
zip_file_name = os.path.basename(zip_file)

print(zip_file)
print(zip_file_name)

\\ambankgroup.ahb.com\GFS\DPM-Performance Management\Historical Data\Raw Data\Retail Analytic\AML\MIS23110545_AML_Monitoring_Report_20231218.zip
MIS23110545_AML_Monitoring_Report_20231218.zip


## OPTION 2: Get latest file

In [13]:
zip_file = all_zips[-1]
zip_file_name = os.path.basename(zip_file)

print(zip_file)
print(zip_file_name)

\\ambankgroup.ahb.com\GFS\DPM-Performance Management\Historical Data\Raw Data\Retail Analytic\AML\MIS23110545_AML_Monitoring_Report_20231218.zip
MIS23110545_AML_Monitoring_Report_20231218.zip


## Unizp

In [14]:
pwd='AML@2023'

with zf.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.setpassword(pwd.encode('utf-8'))
    zip_ref.extractall(INFILE)

xls_file_name = zip_file_name.replace('zip','xlsx')
print(f"Extracting {xls_file_name}")

today_file = glob.glob(os.path.join(INFILE, xls_file_name))[0]
print(f"Running {today_file}")

Extracting MIS23110545_AML_Monitoring_Report_20231218.xlsx
Running \INFILE\MIS23110545_AML_Monitoring_Report_20231218.xlsx


In [15]:
import re

as_at = re.findall(r'(\d{8})\.', today_file)[0]

as_at_date = pd.to_datetime(as_at, format='%Y%m%d')
as_at_db = as_at_date.strftime('%Y-%m-%d')

print(as_at_db)

2023-12-18


In [16]:

new_cols_names = {
    'Business_Dt':'business_date',
    'dcbrncategory':'branch_category',
    'category_new':'channel',
    'Source_Code_Channel':'channel_src_code',
    'Source_Code_Branch':'branch_src_code',
    'Special_Tagging':'special_tagging',
    'SpecialTag':'special_tag',
    'AmWINS_Ref_No':'ref_no',
    'Agent_ID':'staff_id',
    'Applied_Facility_Amount':'facility_amount',
    'Applied_Net_Facility_Amount':'facility_amount_net',
    'Current_Application_Status':'application_status',
    'Reason':'reason',
    'Date_created':'date_created',
    'Disbursement_Date':'date_disbursed',
    'Facility':'facility',
    'Facility_Package':'facility_package',
    'Pricing_Option':'pricing_option',
    'Date_Applied':'date_applied',
    'App_Month':'data_month',
    'Category':'category',
    'workflow_to_status':'workflow_to_status'}

col_dt_type = {
    'App_Month':str, 
    'Agent_ID':str}

df = pd.read_excel(today_file, dtype=col_dt_type, sheet_name='Branches')
df.rename(columns = new_cols_names, inplace=True)


In [17]:
df['fy'] = fy
df['date_as_at'] = as_at_db

# extract branch code
df['branch_code'] = df['branch_src_code'].str[1:4]
col_pr = ['branch_src_code','branch_code']
print(df[col_pr].head(10))

# fix month format
data_month = df['data_month'].str[2:6]
df['data_month'] = df['data_month'].str[2:6]

# add product code
df['prpt_code'] = 'AMNL'
df['remarks'] = 'data_month is application_month'

#print(df.head(3))


                              branch_src_code branch_code
0                                B058 - KULIM         058
1                              B206 - KEMAMAN         206
2                               B104 - TAMPOI         104
3                               B069 - SKUDAI         069
4                                B058 - KULIM         058
5         B039 - MELAKA - JLN MUNSHI ABDULLAH         039
6                                B036 - KULAI         036
7  B006 - AmBank - Melaka (TAMAN MELAKA RAYA)         006
8                        B018 - AmBank - Muar         018
9               B020 - AmBank - Seberang Jaya         020


In [20]:
from urllib.parse import quote  
from sqlalchemy.engine import create_engine




engine = create_engine('mysql+pymysql://rbpadmin:%s@10.12.215.45:3306/prpdb_prod' % quote('rbp0rt@l1'))

tbl_target = "prdt_monitoring_new"

print(f"Table target: {tbl_target}")

Table target: prdt_monitoring_new


In [21]:
# truncate PRDT_monitoring

from sqlalchemy import text

with engine.begin() as connection:
    result = connection.execute(f"TRUNCATE {tbl_target}")
    affected_rows = result.rowcount
    print(f"{affected_rows} rows in {tbl_target} truncated")

0 rows in prdt_monitoring_new truncated


In [22]:
try:
    df.to_sql(tbl_target, con=engine, if_exists='append', index=False)
    print('successfully appended')
except OperationError as e:
    print("OE: ", e)
    print("Error Code: ", e.orig.args[0])

successfully appended


In [23]:
# insert on duplicate update as-at date in PRDC

from sqlalchemy.dialects.mysql import insert

def insert_on_duplicate(table, conn, keys, data_iter):
    insert_stmt = insert(table.table).values(list(data_iter))
    on_duplicate_key_stmt = insert_stmt.on_duplicate_key_update(insert_stmt.inserted)
    conn.execute(on_duplicate_key_stmt)

# Initialize data to lists.
prdc = [{
    'data_type': 'DAY', 
    'data_month': data_month, 
    'code': 'AMNL', 
    'prrt_code': 'PRD_MON',
    'as_at': as_at_db}]
 
# Creates DataFrame.
df_prdc = pd.DataFrame(prdc)
df_prdc.to_sql('prdc_disclaimers', con=engine, if_exists='append', index=False, method=insert_on_duplicate)


In [24]:

qry_name = ["Retag 228->013", "Update region", "OTH region"]

queries = [
    
    # retag 228 - rugayah to 013 - rahmat 
    f"""UPDATE {tbl_target} 
        SET branch_code = '013',
            branch_code_ori = 228
        WHERE fy = '{fy}'
            AND branch_code = 228""",
    
    # region_code
    f"""UPDATE {tbl_target} d, bcus_unit_subunit c
        SET d.region_code = bcus_bcut_code
        WHERE fy = '{fy}'
            AND d.branch_code = bcus_bcsb_code 
            AND bcus_month_start <= d.data_month
            AND (bcus_month_end  IS NULL OR d.data_month <= bcus_month_end)""",
    
    # region_code = OTH
    f"""UPDATE {tbl_target} d
        SET region_code = 'OTH'
        WHERE fy = '{fy}'
            AND region_code IS NULL"""]

idx = 0;
with engine.begin() as connection:
    
    for i in range(len(queries)):
        result = connection.execute(text(queries[i]))
        affected_rows = result.rowcount
        qry_txt = qry_name[i].ljust(45,".")
        #print(f'\n{text(queries[i])}')
        print(f"{qry_txt}: {affected_rows} rows updated")
    
#     for qry in queries:
#         result = connection.execute(text(qry))
#         affected_rows = result.rowcount
#         print(qry_name[idx++])
#         print(f"{affected_rows} rows updated")

print("updates completed")

Retag 228->013...............................: 30 rows updated
Update region................................: 4698 rows updated
OTH region...................................: 38 rows updated
updates completed


# Extract Excel

In [25]:
from datetime import datetime

mth = as_at_date.strftime('%y%m')

with engine.connect() as conn:
    
    qry = f"""
        SELECT COUNT(DISTINCT date_format(date_created,'%%d/%%m/%%Y')) days_occured
        FROM prdt_monitoring_new
        WHERE fy = '{fy}'
            AND data_month = {mth}"""
    rs = conn.execute(qry)
    days_occured = rs.fetchone()[0]
    
    qry = f"""
        SELECT MAX(DISTINCT date_format(date_created,'%%d'))*1 days_total
        FROM prdt_monitoring_new
        WHERE fy = '{fy}'
            AND data_month = {mth}"""
    rs = conn.execute(qry)
    days_total = rs.fetchone()[0]
    

qry = f"""
    SELECT data_month `Month`, 
        date_format(business_date,'%%b-%%y') month_abb, 
        region_code `Region Code`, 
        branch_code `Branch Code`, 
        bcsb_name `Branch Name`,
        staff_id `Staff ID`, 
        bshr_name `Staff Name`,
        bshr_position_name `Position`,
        business_date `Business Date`, 
        ref_no `Case`, 
        facility_amount `Facility Amount`, 
        facility_amount_net `Net Facility Amount`, 
        application_status `Application Status`, category `Category`, 
        workflow_to_status `Workflow Status`
    FROM prdt_monitoring_new
        INNER JOIN bcsb_subunit ON branch_code = bcsb_code
        LEFT JOIN bshr_hr_stafflisting ON (bshr_staff_id = staff_id AND data_month = bshr_month)
    WHERE fy = '{fy}'
    ORDER BY data_month"""

df_data = pd.read_sql(qry, engine)

qry = f"""
    SELECT DISTINCT category
    FROM prdt_monitoring_new
    WHERE fy = '{fy}';"""

df_cat = pd.read_sql(qry, engine)

qry = f"""
    SELECT data_month `Month`, 
        date_format(business_date,'%%b-%%y') month_abb, 
        region_code `Region Code`, 
        IFNULL(bshr_name,'-') `RD`,
        COUNT(ref_no) `Case`, 
        SUM(facility_amount) `Facility Amount`, 
        category `Category`
    FROM prdt_monitoring_new
        LEFT JOIN bshr_hr_stafflisting ON 
            (region_code = bshr_bcut_code 
             AND bshr_bstp_tag = 'RD' 
             AND bshr_month IN (SELECT MAX(bshr_month) FROM bshr_hr_stafflisting))
    WHERE fy = '{fy}'
        AND region_code NOT IN ('OTH')
    GROUP BY data_month, category, region_code
    ORDER BY data_month, category, region_code"""
df_region = pd.read_sql(qry, engine)


qry = f"""
    SELECT DISTINCT bcus_bcut_code `Region Code`, 
        IFNULL(bshr_name,'-') `RD`,
        mtd_disbursed `MTD Disbursed`, 
        mtd_runrate `MTD Runrate`,
        ytd_disbursed `YTD Disbursed`, 
        ytd_runrate `YTD Runrate`
    FROM bcus_unit_subunit
        LEFT JOIN (
            SELECT fy, business_date, region_code, data_month,
                SUM(facility_amount) mtd_disbursed, 
               (SUM(facility_amount)/{days_occured})*{days_total}  mtd_runrate
            FROM prdt_monitoring_new
            WHERE fy = '{fy}'

                AND category LIKE 'Disbursed'
                
                AND data_month = {mth}
            GROUP BY region_code) mtd
        ON mtd.region_code = bcus_bcut_code

        LEFT JOIN (
            SELECT region_code, {mth} data_month,
                SUM(facility_amount) ytd_disbursed, 
               (SUM(facility_amount)/{days_occured})*{days_total}  ytd_runrate
            FROM prdt_monitoring_new
            WHERE fy = '{fy}'
                AND category = 'Disbursed'
                AND data_month <= {mth}
            GROUP BY region_code) ytd
        ON ytd.region_code = bcus_bcut_code

        LEFT JOIN bshr_hr_stafflisting 
            ON (bshr_bcut_code = bcus_bcut_code
                AND bshr_bstp_tag = 'RD' 
                AND bshr_month IN (SELECT MAX(bshr_month) FROM bshr_hr_stafflisting WHERE bshr_month <= {mth}))
    WHERE bcus_cmth_financial_year = '{fy}'
       AND bcus_bcdp_code= 'DIST'
       AND bcus_month_start <= {mth}
       AND (bcus_month_end  IS NULL OR {mth} <= bcus_month_end)
    ORDER BY bcus_bcut_code"""

df_runrate = pd.read_sql(qry, engine)
print('completed')

completed


In [26]:
qry = f"""
SELECT DISTINCT bcus_bcut_code, bshr_name rd
FROM bcus_unit_subunit
    LEFT JOIN bshr_hr_stafflisting ON 
            (bcus_bcut_code = bshr_bcut_code 
                AND bshr_bstp_tag = 'RD' 
                AND bshr_month IN (SELECT MAX(bshr_month) FROM bshr_hr_stafflisting))
WHERE bcus_cmth_financial_year = '{fy}'
    AND bcus_bcdp_code= 'DIST'
    AND bcus_month_start <= {mth}
    AND (bcus_month_end  IS NULL OR {mth} <= bcus_month_end)
"""
df_all_regions = pd.read_sql(qry, engine)
print(df_all_regions)

ls_categories = ['Disbursed', 'Pending', 'Cancelled', 'Rejected']
print(ls_categories)

index = pd.MultiIndex.from_product([df_all_regions['bcus_bcut_code'], df_all_regions['rd'], ls_categories], names=['Region','RD','Status'])
df_dummy = pd.DataFrame(0, index=index, columns=['Case','Facility Amount'])

#print(df_dummy)
#print(df_region)

df_all = pd.concat([df_dummy, df_region], axis=0)
#print(df_all)

  bcus_bcut_code                        rd
0            ECR               LEE SZE WEI
1            JHR             TAN HIANG TAK
2            KV1           CHONG CHIUN YIH
3            KV2             SOO WING HONG
4            MNS  RABI'AH BINTI MOHD TAHIR
5            PNG              LIM KOK BENG
6            PRK              CHIN WAI TAI
7            SBH       JOHN GUANG WEI MING
8            SWK      SIMON SU HIONG TIING
['Disbursed', 'Pending', 'Cancelled', 'Rejected']


In [28]:
from openpyxl import load_workbook

#today_date = datetime.now()
#formatted_date = today_date.strftime('%y%m%d')

rpt_date = as_at_date.strftime('%Y%m%d')
rpt_month = as_at_date.strftime('%d/%m/%Y')

df_mth_map = df_region[['Month','month_abb']].drop_duplicates()
dict_mth_map = dict(zip(df_mth_map['Month'], df_mth_map['month_abb']))

file_name = f'AmMoneyline Daily Monitoring Report as at {rpt_date}.xlsx'
file_working=os.path.join('OUTFILE', file_name)

sort_categories = {'Disbursed':0, 'Pending':1, 'Cancelled':2, 'Rejected':3}

with pd.ExcelWriter(file_working, engine='xlsxwriter') as excel_writer:

    #-----------------------------------------------------------------------------
    ws_name = 'Runrate'
    
    df_runrate.to_excel(excel_writer, sheet_name=ws_name, index=True, startrow=5)
    print(f'{ws_name} - completed')
    
    
    #-----------------------------------------------------------------------------
    ws_name = 'Pipeline'
    piv_region = pd.pivot_table(
        df_region[df_region['Month'] == mth],
        values = ['Case','Facility Amount'],
        index = ['Region Code','RD'],
        columns='Category',
        aggfunc = {'Case':'sum', 'Facility Amount':'sum'},
        fill_value=0,
        margins=True,
        margins_name='Total')
    
    piv_region.to_excel(excel_writer, sheet_name=ws_name, index=True, startrow=2)
    print(f'{ws_name} - completed')
    
    
    #-----------------------------------------------------------------------------
    ws_name = 'MoM Region'
    piv_region = pd.pivot_table(
        df_region,
        values = ['Case','Facility Amount'],
        index = ['Category','Region Code'],
        columns='Month',
        aggfunc = {'Case':'sum', 'Facility Amount':'sum'},
        fill_value=0,
        margins=True,
        margins_name='Total')
    piv_region.rename(columns=dict_mth_map, inplace=True)
    piv_region.to_excel(excel_writer, sheet_name=ws_name, index=True, startrow=2)
    print(f'{ws_name} - completed')
    
    
    #-----------------------------------------------------------------------------
    for cat in df_cat['category']:
        ws_name = f'MoM Branch-{cat}'
        piv_cat = pd.pivot_table(
            df_data[(df_data['Category'] == cat)],
            values = ['Case','Facility Amount'],
            index = ['Region Code','Branch Code','Branch Name'],
            columns='Month',
            aggfunc = {'Case':'count', 'Facility Amount':'sum'},
            fill_value=0,
            margins=True,
            margins_name='Total')
        piv_cat.rename(columns=dict_mth_map, inplace=True)
        piv_cat.to_excel(excel_writer, sheet_name=ws_name, index=True, startrow=2)
        print(f'{cat} - completed')
    
    #-----------------------------------------------------------------------------
    ws_name = 'Data'
    df_data.to_excel(excel_writer, sheet_name=ws_name, index=True, startrow=0)
    print(f'{ws_name} - completed')

wb = load_workbook(file_working)

ws = wb['Runrate']
ws['A1'] = 'Disbursement'  
ws['A2'] = f'Runrate as at {rpt_month}'  

ws['A3'] = 'No. of transaction days '  
ws['B3'] = days_occured  

ws['A4'] = 'No. of total days'   
ws['B4'] = days_total 

ws = wb['Pipeline']
ws['A1'] = 'Submission' 
ws['A2'] = f'Runrate as at {rpt_month}'    

wb.save(file_working)

print(f'Completed writing {file_working}')

Runrate - completed
Pipeline - completed
MoM Region - completed
Rejected - completed
Disbursed - completed
Cancelled - completed
Pending - completed
Data - completed
Completed writing OUTFILE\AmMoneyline Daily Monitoring Report as at 20231218.xlsx


In [29]:
# open generated report & run formating macro

import subprocess

try:
    subprocess.Popen(['start', 'excel', file_working], shell=True)
    print(f'{file_working} is opened.')
except FileNotFoundError:
    print(f'{file_working} was not found.')
except Exception as e:
    print(f'Error: {e}')

OUTFILE\AmMoneyline Daily Monitoring Report as at 20231218.xlsx is opened.


In [30]:
#copy formatted xls to end-users' report folder

import shutil

REPORT_FOLDER = r'\\ambankgroup.ahb.com\GFS\DPM-Performance Management\Historical Data\PRP-DM Output\Reports\Daily AmMoneyline Monitoring'
#REPORT_FOLDER = r'C:\Users\10033084B\Documents\prp-python\daily\AMNL_Monitoring\OUTFILE\TGT_RPT'

file_report=os.path.join(REPORT_FOLDER, file_name)

shutil.copy(file_working, file_report)

'\\\\ambankgroup.ahb.com\\GFS\\DPM-Performance Management\\Historical Data\\PRP-DM Output\\Reports\\Daily AmMoneyline Monitoring\\AmMoneyline Daily Monitoring Report as at 20231218.xlsx'

In [31]:
#open copied formatted xls from end-users' report folder - for checking
#close working file first!

try:
    subprocess.Popen(['start', 'excel', file_report], shell=True)
    print(f'{file_working} is opened.')
except FileNotFoundError:
    print(f'{file_working} was not found.')
except Exception as e:
    print(f'Error: {e}')

OUTFILE\AmMoneyline Daily Monitoring Report as at 20231218.xlsx is opened.


# PRDS - WIP
## RR

In [ ]:
#config target

from sqlalchemy import text


In [ ]:
from sqlalchemy import text
print(mth)


tbl_target = 'prds_daily_summary'
with engine.begin() as connection:
    result = connection.execute(f"DELETE FROM {tbl_target} WHERE data_month = {mth} AND report_code = 'AML_RR'")
    affected_rows = result.rowcount
    print(f"{affected_rows} rows in {tbl_target} deleted")
    
cols = ['MTD Disbursed','MTD Runrate','YTD Disbursed','YTD Runrate','Monthly Target','MTD Target']
keys = ['MTD_DIS','MTD_RR','YTD_DIS','YTD_RR','MTH_TGT','MTD_TGT']

keys_dict = {
    'MTD_DIS': 'MTD Disbursed',
    'MTD_RR': 'MTD Runrate',
    'YTD_DIS': 'YTD Disbursed',
    'YTD_RR': 'YTD Runrate',
}

for k, col in keys_dict.items():
    print('\n')
    print(k)
    cols = ['Region Code', col]
    df_prds = df_runrate[cols]

    df_prds['data_month'] = mth
    df_prds['report_code'] = 'AML_RR'
    
    df_prds = df_prds.rename(
        columns={
            'Region Code': 'data_key', 
            col: 'data_val'})
    
    df_prds['data_key'] = df_prds['data_key'].astype(str) + f'_{k}'
    
    print(df_prds)
    
    try:
        df_prds.to_sql(tbl_target, con=engine, if_exists='append', index=False)
        print('successfully appended')
    except OperationError as e:
        print("OE: ", e)
        print("Error Code: ", e.orig.args[0])


tgt_dict = {
    'ECR': '650000',
    'JHR': '2000000',
    'KV1': '2000000',
    'KV2': '2000000',
    'MNS': '2000000',
    'PNG': '500000',
    'PRK': '1900000',
    'SBH': '550000',
    'SWK': '800000'
}

df_tgt = pd.DataFrame(columns=['data_month', 'report_code', 'data_key', 'data_val'])

new_row = {
    'data_month': mth, 
    'report_code': 'AML_RR', 
    'data_key': 'DAY_TOTAL', 
    'data_val': days_total}
df_tgt = df_tgt.append(new_row, ignore_index=True)

new_row = {
    'data_month': mth, 
    'report_code': 'AML_RR', 
    'data_key': 'DAY_OCCUR', 
    'data_val': days_occured}
df_tgt = df_tgt.append(new_row, ignore_index=True)

for rg, tgt in tgt_dict.items():
    
    new_row = {
        'data_month': mth, 
        'report_code': 'AML_RR', 
        'data_key': rg + '_MTH_TGT', 
        'data_val': tgt}
    df_tgt = df_tgt.append(new_row, ignore_index=True)
    
    mtd_tgt = (float(tgt)/float(days_total))*float(days_occured)
    new_row = {
        'data_month': mth, 
        'report_code': 'AML_RR', 
        'data_key': rg + '_MTD_TGT', 
        'data_val': mtd_tgt}
    df_tgt = df_tgt.append(new_row, ignore_index=True)
    
print(df_tgt)


try:
    df_tgt.to_sql(tbl_target, con=engine, if_exists='append', index=False)
    print('successfully appended')
except OperationError as e:
    print("OE: ", e)
    print("Error Code: ", e.orig.args[0])


## PL